In [2]:
import tensorflow as tf
import skimage.io
import skimage.color
import numpy as np
from pprint import pprint

In [3]:
import read_cifar10 as cf10

#@read_data.restartable
def cifar10_dataset_generator(dataset_name, batch_size, restrict_size=1000):
    assert dataset_name in ['train', 'test']
    assert batch_size > 0 or batch_size == -1  # -1 for entire dataset
    
    X_all_unrestricted, y_all = (cf10.load_training_data() if dataset_name == 'train'
                                 else cf10.load_test_data())
    
    actual_restrict_size = restrict_size if dataset_name == 'train' else int(1e10)
    X_all = X_all_unrestricted[:actual_restrict_size]
    data_len = X_all.shape[0]
    batch_size = batch_size if batch_size > 0 else data_len
    
    X_all_padded = np.concatenate([X_all, X_all[:batch_size]], axis=0)
    y_all_padded = np.concatenate([y_all, y_all[:batch_size]], axis=0)
    
    for slice_i in range(math.ceil(data_len / batch_size)):
        idx = slice_i * batch_size
        #X_batch = X_all_padded[idx:idx + batch_size]
        X_batch = X_all_padded[idx:idx + batch_size]*255  # bugfix: thanks Zezhou Sun!
        y_batch = np.ravel(y_all_padded[idx:idx + batch_size])
        yield X_batch.astype(np.uint8), y_batch.astype(np.uint8)

cifar10_dataset_generators = {
    'train': cifar10_dataset_generator('train', 1000),
    'test': cifar10_dataset_generator('test', -1)
}


In [4]:
#Load cifar-10 data
cf10_tr=cf10.load_training_data()
cf10_tr_img=cf10_tr[0]
cf10_tr_label = cf10_tr[1]

cf10_test=cf10.load_test_data()
cf10_test_img=cf10_test[0]
cf10_test_label = cf10_test[1]

In [6]:
#Calculate the metrics for jpg
mse_jpg=np.zeros((10000,1))
psnr_jpg=np.zeros((10000,1))
#Save cifar test images
for k in range(10000):
    img_gray=(255*cf10_test_img_gray[k,:,:]).astype(np.uint8)
    skimage.io.imsave('../cifar10_jpg/'+str(k)+'.jpg',img_gray)
    
    img_vec = img_gray.reshape([1,-1])
    cf10_test_vec[k,:]=img_vec
    img_recons = skimage.io.imread('../cifar10_jpg/'+str(k)+'.jpg')
    mse=(((img_recons.astype(float)-img_gray.astype(float))/255.)**2).mean()
    mse_jpg[k]=mse
    psnr_jpg[k]=10.*np.log10(1./mse)

print('test is done')
for k in range(50000):
    img_gray=(255*cf10_tr_img_gray[k,:,:]).astype(np.uint8)
    img_vec = img_gray.reshape([1,-1])
    cf10_tr_vec[k,:]=img_vec
    
print('train is done')



NameError: name 'cf10_test_img_gray' is not defined

In [7]:
#Not necessary
print(mse_jpg.mean())
print(psnr_jpg.mean())

0.0
0.0


In [4]:
import skimage.io
import numpy as np
def img2block(im):
    #lion[:2816,:4224,:]
    #skimage.io.imshow(lion)
    #skimage.io.show()
    
    im = im.astype(np.float32)
    row,col,color = im.shape
    im_bl=np.zeros((int(row*col/1024),32,32,3)).astype(np.float32)
    count=0
    for i in range(0,row-row%32,32):
        for j in range(0,col-col%32,32):
            im_bl[count,:,:,:]=im[i:i+32,j:j+32,:]
            count = count +1
    im_bl=im_bl/255.
    return im_bl

def block2img(img_blocks,img_size):
    
    row,col = img_size
    img=np.zeros((row,col,3)).astype(np.float32)
    n,k,l,c=img_blocks.shape
                 
    for i in range(0,int(row/k)):
        for j in range(0,int(col/k)):
            img[i*k:(i+1)*k,j*l:(j+1)*l,:]=img_blocks[int(i*col/k+j),:,:,:]
    return img

lena_img = skimage.io.imread('../test_img_old/lena512color.tiff')

lena_32=img2block(lena_img)


'''
bu_img_recon=block2img(bu_img_32,(672,1024))
bu_img_recon = (bu_img_recon*255.).astype(np.uint8)
skimage.io.imshow(bu_img)
skimage.io.show()
skimage.io.imshow(bu_img_recon)
skimage.io.show()
#skimage.io.imsave('../test_img/bu2010_recon.tif',bu_img_recon)
''''      
        

NameError: name 'bu_img_32' is not defined

In [6]:
def convert2uint8(img):
    img[img>255]=255
    img[img<0]=0
    return img.astype(np.uint8)

# Part-2 - CNN

In [7]:
x_tr = cf10_tr_img.astype(np.float32)#*255.
x_test = cf10_test_img.astype(np.float32)#*255.
x_test=x_test[:200,:,:,:]
img = skimage.io.imread('../test_img_old/lena512color.tiff')
img_32=img2block(img)
#print(lion_32.shape)
outname='/tmp/cnn_autoencoder'
a=outname+'FT'
print(outname)
print(a)

/tmp/cnn_autoencoder
/tmp/cnn_autoencoderFT


In [8]:
def cnn_autoencoder(x_,kernels1=[5,7],kernels2=[7,5],filters1=[16,128],filters2=[128,3],pool_size=[1,2,2,1]):
    out_=x_
    for k in range(len(kernels1)):
        conv = tf.layers.conv2d(inputs=out_,
                                filters=filters1[k],
                                kernel_size=[kernels1[k],kernels1[k]],
                                padding="same",
                                activation=tf.nn.relu,
                                name='conv'+str(k))
        pool_now=pool_size[k]
        if(pool_now==1):
            out_=conv
        else:
            out_ = tf.layers.max_pooling2d(inputs=conv, 
                                           pool_size=[pool_now,pool_now], 
                                           strides=pool_now,
                                           name = 'pool'+str(k))
        
        out_quant=tf.round(out_*255.)/255.

    for k in range(len(kernels2)):
        with tf.variable_scope("deconv") as var_scope:
            pool_now=pool_size[k+len(kernels1)]
            if(pool_now==1):
                x_up=out_
                out_ = tf.layers.conv2d(inputs=x_up,
                                        filters=filters2[k],
                                        kernel_size=[kernels2[k],kernels2[k]],
                                        padding="same",
                                        activation=tf.nn.relu,
                                        name='deconv'+str(k))
                var_scope.reuse_variables() 
                x_quant_up=out_quant
                out_quant = tf.layers.conv2d(inputs=x_quant_up,
                                            filters=filters2[k],
                                            kernel_size=[kernels2[k],kernels2[k]],
                                            padding="same",
                                            activation=tf.nn.relu,
                                            name='deconv'+str(k))
            else:
                sh = out_.get_shape().as_list()
                x_up=tf.image.resize_images(out_,[sh[1]*pool_now,sh[2]*pool_now])
                out_ = tf.layers.conv2d(inputs=x_up,
                                        filters=filters2[k],
                                        kernel_size=[kernels2[k],kernels2[k]],
                                        padding="same",
                                        activation=tf.nn.relu,
                                        name='deconv'+str(k))
                var_scope.reuse_variables() 
                x_quant_up=tf.image.resize_images(out_quant,[sh[1]*pool_now,sh[2]*pool_now])
                out_quant = tf.layers.conv2d(inputs=x_quant_up,
                                            filters=filters2[k],
                                            kernel_size=[kernels2[k],kernels2[k]],
                                            padding="same",
                                            activation=tf.nn.relu,
                                            name='deconv'+str(k))


    return out_,out_quant


In [12]:
#HVS matrix implementation
import scipy.fftpack as fft
f=np.linspace(1.0, 40.0, num=32)
h=2.6*(0.0192+0.114*f)*np.exp(-np.power((0.114*f),1.1)).reshape([1,-1])
H=np.matmul(np.ones((32,1)),h)
H2=(H*H.T).astype(np.float32)
print(H2)
h2 = fft.dct(fft.dct(H2,type=3,norm='ortho').T,type=3,norm='ortho')
h3=np.zeros((32,32,3,1)).astype(np.float32)
h3[:,:,0,0]=h2
h3[:,:,1,0]=h2
h3[:,:,2,0]=h2


print(h2)
print(h3)

[[ 0.09983069  0.18150099  0.23934336 ...,  0.02520034  0.02169322
   0.01864358]
 [ 0.18150099  0.32998478  0.43514732 ...,  0.04581644  0.03944018
   0.03389566]
 [ 0.23934336  0.43514732  0.57382399 ...,  0.06041763  0.05200934
   0.04469784]
 ..., 
 [ 0.02520034  0.04581644  0.06041763 ...,  0.00636134  0.00547604
   0.00470621]
 [ 0.02169322  0.03944018  0.05200934 ...,  0.00547604  0.00471394
   0.00405125]
 [ 0.01864358  0.03389566  0.04469784 ...,  0.00470621  0.00405125
   0.00348172]]
[[  1.00616789e+01   1.49537575e+00  -8.23408246e-01 ...,  -1.08088404e-02
   -6.29915297e-03  -7.97748566e-03]
 [  1.49537563e+00   2.22244024e-01  -1.22375660e-01 ...,  -1.60639174e-03
   -9.36212018e-04  -1.18559599e-03]
 [ -8.23408127e-01  -1.22375637e-01   6.73845112e-02 ...,   8.84542242e-04
    5.15500084e-04   6.52849674e-04]
 ..., 
 [ -1.08086262e-02  -1.60637801e-03   8.84536712e-04 ...,   1.16189622e-05
    6.77026401e-06   8.54581594e-06]
 [ -6.29931595e-03  -9.36207653e-04   5.15490

In [9]:
#Working
def apply_classification_loss_mse(kernels1=[5,7],kernels2=[7,5],
                                 filters1=[16,128],filters2=[128,3],
                                pool_size=[2,2],learning_rate=1.,FT=False):
    with tf.Graph().as_default() as g:
        with tf.device("/gpu:0"):  # use gpu:0 if on GPU
            x_ = tf.placeholder(tf.float32, [None, 32, 32, 3])
            (x_out,x_out_quant)=cnn_autoencoder(x_,pool_size=pool_size,kernels1=kernels1,filters1=filters1,
                                kernels2=kernels2,filters2=filters2)

            #y_dict = dict(labels=y_, logits=y_logits)
            #losses = tf.nn.sparse_softmax_cross_entropy_with_logits(**y_dict)
            #cross_entropy_loss = tf.reduce_mean(losses)
            mse_loss1=tf.reduce_mean(tf.subtract(x_,x_out)**2)
            #a=tf.pad(tf.subtract(x_,x_out),[[0,0],[16,16],[16,16],[0,0]],'CONSTANT')

            #mse_loss1=tf.reduce_mean(tf.nn.conv2d(a,h3,strides=[1,1,1,1],padding="VALID")**2)
            mse_loss2=tf.reduce_mean(tf.subtract(x_,x_out_quant)**2)
            trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            if(FT):
                with tf.variable_scope('deconv', reuse=True) as vs:
                    var_list=[v for v in tf.global_variables() if v.name.startswith(vs.name)]
                train_op = trainer.minimize(mse_loss1,var_list=var_list)
            else:
                train_op = trainer.minimize(mse_loss1)

            #y_pred = tf.argmax(tf.nn.softmax(y_logits), dimension=1)
            #correct_prediction = tf.equal(tf.cast(y_pred, tf.int32), y_)
            #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    model_dict = {'graph': g, 'inputs': x_,'outputs':x_out_quant, 'train_op': train_op, 'loss1': mse_loss1,'loss2': mse_loss2}
    
    return model_dict

In [29]:
#In progress
def apply_classification_loss_ssim(kernels1=[5,7],kernels2=[7,5],
                                 filters1=[16,128],filters2=[128,3],
                                pool_size=[2,2],learning_rate=1.):
    with tf.Graph().as_default() as g:
        with tf.device("/gpu:0"):  # use gpu:0 if on GPU
            x_ = tf.placeholder(tf.float32, [None, 32, 32, 3])
            (x_out,x_out_quant)=cnn_autoencoder(x_,pool_size=pool_size,kernels1=kernels1,filters1=filters1,
                                kernels2=kernels2,filters2=filters2)

            #y_dict = dict(labels=y_, logits=y_logits)
            #losses = tf.nn.sparse_softmax_cross_entropy_with_logits(**y_dict)
            #cross_entropy_loss = tf.reduce_mean(losses)
            mse_loss1=tf.reduce_mean(tf.subtract(x_,x_out)**2)
            #a=tf.pad(tf.subtract(x_,x_out),[[0,0],[16,16],[16,16],[0,0]],'CONSTANT')

            #mse_loss1=tf.reduce_mean(tf.nn.conv2d(a,h3,strides=[1,1,1,1],padding="VALID")**2)
            mse_loss2=tf.reduce_mean(tf.subtract(x_,x_out_quant)**2)
            trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            train_op = trainer.minimize(mse_loss1)

            #y_pred = tf.argmax(tf.nn.softmax(y_logits), dimension=1)
            #correct_prediction = tf.equal(tf.cast(y_pred, tf.int32), y_)
            #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    model_dict = {'graph': g, 'inputs': x_,'outputs':x_out_quant, 'train_op': train_op, 'loss1': mse_loss1,'loss2': mse_loss2}
    
    return model_dict

In [10]:
def train_model(model_dict, dataset_generators, train_every=100, test_every=200, load=False,
                learning_rate=1.,fname='cifar10_recon',outname='/tmp/cnn_autoencoder',ftname='/tmp/cnn_autoencoder'):
    with model_dict['graph'].as_default(), tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        
        
        
        
        ### WORK ON IT NOT TOO HARD
        saver=tf.train.Saver()
        if(load):
            saver.restore(sess, outname)
            print("Model loaded")
        else:
            sess.run(tf.global_variables_initializer())
        
        ids=[i for i in range(100)]
        for iter_i in range(100000):
            batch_xs = x_tr[ids,:,:,:] 
            ids=[(ids[0]+100+i)%x_tr.shape[0] for i in range(100)]
            sess.run(model_dict['train_op'], feed_dict={model_dict['inputs']: batch_xs})
            
            # test trained model
            if iter_i % train_every == 0:
                tf_feed_dict = {model_dict['inputs']: batch_xs}
                loss_val = sess.run(model_dict['loss1'], feed_dict={model_dict['inputs']: batch_xs})
                print('iteration %d\t train mse: %.3E\t'%(iter_i,loss_val))
                if iter_i % test_every == 0:
                    #tf_feed_dict = {x_: x_test}
                    loss_val1 = sess.run(model_dict['loss1'], feed_dict={model_dict['inputs']: x_test})
                    loss_val2 = sess.run(model_dict['loss2'], feed_dict={model_dict['inputs']: x_test})
                    print('iteration %d\t TEST MSE: %.3E\t TEST MSE(Quantized): %.3E\t'%(iter_i,loss_val1,loss_val2))
                    
                    
                    lena_block=sess.run(model_dict['outputs'], 
                                       feed_dict={model_dict['inputs']:lena_32})
                    x_from_test=sess.run(model_dict['outputs'], 
                                         feed_dict={model_dict['inputs']:x_test[:5,:,:,:].reshape([-1,32,32,3])})
                    
                    lena_recon=block2img(lena_block,(512,512))
                    lena_recon = convert2uint8(lena_recon*255.)
                    skimage.io.imsave('../'+fname+'/lena4_'+str(int(iter_i/test_every))+'.tiff',lena_recon)

                    for i in range(2):
                        img_recon=convert2uint8((255*x_from_test[i,:,:,:]).reshape([32,32,3])).astype(np.uint8)
                        skimage.io.imsave('../'+fname+'/test'+str(i)+'_'+str(int(iter_i/test_every))+'.tiff',img_recon)
                        
        saver = tf.train.Saver()
        if(load):
            outname=ftname
        save_path = saver.save(sess, outname)
        print("Model saved in file: %s" % save_path)
        return saver
                


In [ ]:
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[64,16,3],filters2=[3,3,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=7e-5)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon0',outname='/tmp/cnnx4_test0')

In [15]:
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[64,16,3],filters2=[64,16,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=7e-5)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon0',outname='/tmp/cnnx4_test0')

iteration 0	 train mse: 2.062E-01	
iteration 0	 TEST MSE: 2.353E-01	 TEST MSE(Quantized): 2.353E-01	
iteration 100	 train mse: 1.615E-02	
iteration 200	 train mse: 1.129E-02	
iteration 300	 train mse: 1.005E-02	
iteration 400	 train mse: 8.972E-03	
iteration 500	 train mse: 8.619E-03	
iteration 600	 train mse: 6.659E-03	
iteration 700	 train mse: 6.726E-03	
iteration 800	 train mse: 7.170E-03	
iteration 900	 train mse: 6.769E-03	
iteration 1000	 train mse: 7.051E-03	
iteration 1100	 train mse: 6.008E-03	
iteration 1200	 train mse: 5.561E-03	
iteration 1300	 train mse: 6.105E-03	
iteration 1400	 train mse: 5.891E-03	
iteration 1500	 train mse: 6.192E-03	
iteration 1600	 train mse: 4.952E-03	
iteration 1700	 train mse: 4.846E-03	
iteration 1800	 train mse: 5.463E-03	
iteration 1900	 train mse: 5.241E-03	
iteration 2000	 train mse: 5.799E-03	
iteration 2000	 TEST MSE: 5.326E-03	 TEST MSE(Quantized): 5.326E-03	
iteration 2100	 train mse: 4.494E-03	
iteration 2200	 train mse: 4.416E-03	
ite

In [13]:
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,5,5],kernels2=[5,5,5],
                                     filters1=[16,20,20],filters2=[20,16,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon1',outname='/tmp/cnnx4_test1')

iteration 0	 train mse: 2.606E-01	
iteration 0	 TEST MSE: 2.958E-01	 TEST MSE(Quantized): 2.958E-01	


/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon1/lena4_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon1/test0_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon1/test1_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)


iteration 100	 train mse: 3.853E-02	
iteration 200	 train mse: 1.683E-02	
iteration 300	 train mse: 1.382E-02	
iteration 400	 train mse: 1.197E-02	
iteration 500	 train mse: 1.148E-02	
iteration 600	 train mse: 8.342E-03	
iteration 700	 train mse: 8.427E-03	
iteration 800	 train mse: 8.820E-03	
iteration 900	 train mse: 8.480E-03	
iteration 1000	 train mse: 8.605E-03	
iteration 1100	 train mse: 6.760E-03	
iteration 1200	 train mse: 6.975E-03	
iteration 1300	 train mse: 7.534E-03	
iteration 1400	 train mse: 7.417E-03	
iteration 1500	 train mse: 7.625E-03	
iteration 1600	 train mse: 6.095E-03	
iteration 1700	 train mse: 6.261E-03	
iteration 1800	 train mse: 6.843E-03	
iteration 1900	 train mse: 6.791E-03	
iteration 2000	 train mse: 7.021E-03	
iteration 2000	 TEST MSE: 6.224E-03	 TEST MSE(Quantized): 6.225E-03	
iteration 2100	 train mse: 5.702E-03	
iteration 2200	 train mse: 5.831E-03	
iteration 2300	 train mse: 6.413E-03	
iteration 2400	 train mse: 6.383E-03	
iteration 2500	 train mse: 6

In [14]:
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,5,5],kernels2=[5,5,5],
                                     filters1=[64,64,64],filters2=[64,64,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon2',outname='/tmp/cnnx4_test2')

iteration 0	 train mse: 2.264E-01	
iteration 0	 TEST MSE: 2.577E-01	 TEST MSE(Quantized): 2.577E-01	


/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon2/lena4_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon2/test0_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon2/test1_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)


iteration 100	 train mse: 1.202E-02	
iteration 200	 train mse: 8.695E-03	
iteration 300	 train mse: 7.953E-03	
iteration 400	 train mse: 7.018E-03	
iteration 500	 train mse: 6.768E-03	
iteration 600	 train mse: 5.333E-03	
iteration 700	 train mse: 5.125E-03	
iteration 800	 train mse: 5.486E-03	
iteration 900	 train mse: 5.352E-03	
iteration 1000	 train mse: 5.343E-03	
iteration 1100	 train mse: 4.352E-03	
iteration 1200	 train mse: 4.276E-03	
iteration 1300	 train mse: 4.830E-03	
iteration 1400	 train mse: 4.678E-03	
iteration 1500	 train mse: 4.669E-03	
iteration 1600	 train mse: 3.853E-03	
iteration 1700	 train mse: 3.780E-03	
iteration 1800	 train mse: 4.318E-03	
iteration 1900	 train mse: 4.352E-03	
iteration 2000	 train mse: 4.221E-03	
iteration 2000	 TEST MSE: 3.847E-03	 TEST MSE(Quantized): 3.847E-03	
iteration 2100	 train mse: 3.531E-03	
iteration 2200	 train mse: 3.429E-03	
iteration 2300	 train mse: 3.944E-03	
iteration 2400	 train mse: 3.842E-03	
iteration 2500	 train mse: 3

In [15]:
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[3,3,3],kernels2=[3,3,3],
                                     filters1=[64,64,64],filters2=[64,64,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon3',outname='/tmp/cnnx4_test3')

iteration 0	 train mse: 2.424E-01	
iteration 0	 TEST MSE: 2.749E-01	 TEST MSE(Quantized): 2.749E-01	


/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon3/lena4_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon3/test0_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon3/test1_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)


iteration 100	 train mse: 2.164E-02	
iteration 200	 train mse: 9.924E-03	
iteration 300	 train mse: 8.607E-03	
iteration 400	 train mse: 7.657E-03	
iteration 500	 train mse: 7.297E-03	
iteration 600	 train mse: 5.629E-03	
iteration 700	 train mse: 5.594E-03	
iteration 800	 train mse: 5.953E-03	
iteration 900	 train mse: 5.774E-03	
iteration 1000	 train mse: 5.817E-03	
iteration 1100	 train mse: 4.761E-03	
iteration 1200	 train mse: 4.738E-03	
iteration 1300	 train mse: 5.109E-03	
iteration 1400	 train mse: 5.022E-03	
iteration 1500	 train mse: 5.077E-03	
iteration 1600	 train mse: 4.176E-03	
iteration 1700	 train mse: 4.134E-03	
iteration 1800	 train mse: 4.513E-03	
iteration 1900	 train mse: 4.464E-03	
iteration 2000	 train mse: 4.525E-03	
iteration 2000	 TEST MSE: 4.093E-03	 TEST MSE(Quantized): 4.095E-03	
iteration 2100	 train mse: 3.741E-03	
iteration 2200	 train mse: 3.698E-03	
iteration 2300	 train mse: 4.110E-03	
iteration 2400	 train mse: 4.042E-03	
iteration 2500	 train mse: 4

In [16]:
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[7,7,7],kernels2=[7,7,7],
                                     filters1=[64,64,64],filters2=[64,64,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon4',outname='/tmp/cnnx4_test4')

iteration 0	 train mse: 2.205E-01	
iteration 0	 TEST MSE: 2.515E-01	 TEST MSE(Quantized): 2.515E-01	


/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon4/lena4_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon4/test0_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)


iteration 100	 train mse: 1.264E-02	
iteration 200	 train mse: 9.381E-03	
iteration 300	 train mse: 1.074E-02	
iteration 400	 train mse: 7.770E-03	
iteration 500	 train mse: 7.581E-03	
iteration 600	 train mse: 5.833E-03	
iteration 700	 train mse: 5.585E-03	
iteration 800	 train mse: 6.067E-03	
iteration 900	 train mse: 5.922E-03	
iteration 1000	 train mse: 5.778E-03	
iteration 1100	 train mse: 4.707E-03	
iteration 1200	 train mse: 4.767E-03	
iteration 1300	 train mse: 5.057E-03	
iteration 1400	 train mse: 5.011E-03	
iteration 1500	 train mse: 5.107E-03	
iteration 1600	 train mse: 4.392E-03	
iteration 1700	 train mse: 4.123E-03	
iteration 1800	 train mse: 4.582E-03	
iteration 1900	 train mse: 5.152E-03	
iteration 2000	 train mse: 4.521E-03	
iteration 2000	 TEST MSE: 4.098E-03	 TEST MSE(Quantized): 4.099E-03	
iteration 2100	 train mse: 3.864E-03	
iteration 2200	 train mse: 3.852E-03	
iteration 2300	 train mse: 4.068E-03	
iteration 2400	 train mse: 4.181E-03	
iteration 2500	 train mse: 4

In [1]:
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,5,5,5,5],kernels2=[5,5,5,5,5],
                                     filters1=[16,32,64,64,128],filters2=[64,64,32,16,3],
                                     pool_size=[1,2,1,2,1,1,2,1,2,1],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon5',outname='/tmp/cnnx4_test5')

NameError: name 'tf' is not defined

In [32]:
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[16,64,128],filters2=[64,16,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon6',outname='/tmp/cnnx4_test6')

NameError: name 'apply_classification_loss' is not defined

In [114]:
img = skimage.io.imread('../test_img_old/lion.tiff')
img_32=img2block(img)

tmp=x_tr.copy()
x_tr=img_32.copy()
idx=np.random.permutation(x_tr.shape[0])
x_tr=x_tr[idx,:,:,:]

tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[16,64,128],filters2=[64,16,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=7e-5,FT=True)
tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,7,9],kernels2=[5,5],
                                     filters1=[16,64,128],filters2=[16,3],
                                     pool_size=[1,2,2,2,2],learning_rate=0.0001,FT=False)
saver = train_model(model_dict, [], train_every=100, test_every=1000,load=True,
                    fname='cifar10_recon0',outname='/tmp/cnn_autoencoderx16',ftname='/tmp/cnnx4_test0')

x_tr=tmp.copy()

INFO:tensorflow:Restoring parameters from /tmp/cnn_autoencoderx16
Model loaded
iteration 0	 train mse: 6.739E-04	
iteration 0	 TEST MSE: 3.041E-03	 TEST MSE(Quantized): 3.042E-03	
iteration 100	 train mse: 4.539E-04	
iteration 200	 train mse: 5.587E-04	
iteration 300	 train mse: 5.086E-04	
iteration 400	 train mse: 6.164E-04	
iteration 500	 train mse: 5.478E-04	
iteration 600	 train mse: 6.552E-04	
iteration 700	 train mse: 5.413E-04	
iteration 800	 train mse: 5.983E-04	
iteration 900	 train mse: 4.947E-04	
iteration 1000	 train mse: 5.851E-04	
iteration 1000	 TEST MSE: 2.817E-03	 TEST MSE(Quantized): 2.818E-03	
iteration 1100	 train mse: 5.438E-04	
iteration 1200	 train mse: 4.355E-04	
iteration 1300	 train mse: 6.725E-04	
iteration 1400	 train mse: 4.570E-04	
iteration 1500	 train mse: 6.676E-04	
iteration 1600	 train mse: 4.150E-04	
iteration 1700	 train mse: 6.416E-04	
iteration 1800	 train mse: 3.885E-04	
iteration 1900	 train mse: 4.092E-04	
iteration 2000	 train mse: 4.575E-04	


In [19]:
#model_dict=apply_classification_loss_mse(kernels1=[5,7,9],kernels2=[5,5],
#                                     filters1=[16,64,128],filters2=[16,3],
#                                     pool_size=[1,2,2,2,2],learning_rate=0.0001,FT=False)

tf.reset_default_graph()
model_dict=apply_classification_loss_mse(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[16,64,128],filters2=[64,16,3],
                                     pool_size=[1,2,2,2,2,1],learning_rate=7e-5)
tfsave ='/tmp/cnnx4_test0'
imgpath = '../test_img_old/lion.tiff'
outpath='../test_img/lion_recon2_convplusx8.tiff'
with model_dict['graph'].as_default(), tf.Session() as sess:
#with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    '''
    var_list=[]
    with tf.variable_scope('conv1',reuse=True):
        var_list.append ( tf.get_variable('kernel') )
    '''
    saver = tf.train.Saver()
    saver.restore(sess, tfsave)
    print("Model loaded")
    
    img = skimage.io.imread(imgpath)
    [w,l,c]=img.shape
    img_32=img2block(img)
    img_block = np.zeros(img_32.shape)
    
    n= np.floor(img_32.shape[0]/2000).astype(int)
    print(n)
    for i in range(0,n):
        print(str(i+1)+'th slice')
        img_block[i*2000:(i+1)*2000,:,:,:]=sess.run(model_dict['outputs'], 
                                    feed_dict={model_dict['inputs']:img_32[i*2000:(i+1)*2000,:,:,:]})
    img_block[n*2000:,:,:,:]=sess.run(model_dict['outputs'], 
                                    feed_dict={model_dict['inputs']:img_32[n*2000:,:,:,:]})
        
    img_recon=block2img(img_block,(w,l))
    img_recon = convert2uint8(img_recon*255.)
    skimage.io.imsave(outpath,img_recon)


INFO:tensorflow:Restoring parameters from /tmp/cnnx4_test0
Model loaded
5
1th slice
2th slice
3th slice
4th slice
5th slice
